see whatsapp messages for possible improvements

get curve for delta

# import data and libraries

In [1]:
using CSV, DataFrames, Plots, Random, DecisionTree, Statistics, LinearAlgebra, JuMP, Gurobi, NearestNeighbors

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [3]:
# load data, we have to keep the data for all the players
data = CSV.read("../data/fifa/players_22_preprocessed.csv", DataFrame)
transfers = CSV.read("../data/Transfers/all_transfers.csv", DataFrame);

In [5]:
# iterate over all clubs, and save players that are part of that clubs
club_players = Dict()
for club in Set(data[:, "club_name"])
    club_players[club] = data[data[:, "club_name"] .== club, :]
end

In [6]:
# keep only 2022 transfers
transfers = transfers[transfers[:, "year"] .== 2022, :];

# Choose club we are making prescription for

When we do prescription, we are pretending to be one specific club. We can do the same for multiple clubs.

In [13]:
club_choosen = "AC Milan"
club_choosen_transferName = "AC Milan"
;

In [14]:
data_club = club_players[club_choosen]
data_rest = data[data[:, "club_name"] .!= club_choosen, :]
;

In [15]:
# get parameters we need in our prescriptive model
function get_parameters(data_club, data_rest, transfers_club)
    n = size(data_rest, 1) # players that can be recruited
    N_c = k = size(data_club, 1) # number of players in the club, equal to how many we can consider to sell
    N_min = minimum([size(club_players[club])[1] for club in keys(club_players)]) # minimum number of players in the club
    N_max = maximum([size(club_players[club])[1] for club in keys(club_players)]) # maximum number of players in the club
    B_max = 10 # maximum number of players that can be bought
    S_max = 3 # maximum number of players that can be sold

    # for players outside the club
    p_o = data_rest[:, "potential"] # potential
    r_o = data_rest[:, "overall"] # rating
    v_o = data_rest[:, "value_eur"] # value (later we will use the value predicted in the prediction part)
    m_o = 0.9 * r_o + 0.1 * p_o # overall rating
    a_o = data_rest[:, "age"] # age

    # for players inside the club
    p_c = data_club[:, "potential"] # potential 
    r_c = data_club[:, "overall"] # rating 
    v_c = data_club[:, "value_eur"] # value (same as above)
    m_c = 0.6 * r_c + 0.4 * p_c # overall rating
    a_c = data_club[:, "age"] # age

    𝛅 = 1 # how much we want to improve the team
    𝛾 = 0.2 # how much we can increase the average age at most

    return n, N_c, N_min, N_max, B_max, S_max, p_o, r_o, v_o, m_o, a_o, p_c, r_c, v_c, m_c, a_c, 𝛅, 𝛾
end

get_parameters (generic function with 1 method)

### interesting idea:

instead of manually choosing the value of 𝛅, we can compute it from the last transfer window of that team.
As a consquence, our optimization model will find the cheapest way to do better than what the team did in real life.
Of course, we can't do that in the future, but it's still true that a team could put the delta they want and get the results.

## Define functions that will be used across models

In [16]:
# define target and features
target = "fee";

In [17]:
function get_features(df)
    return select(df, Not(target))
end

get_features (generic function with 1 method)

In [18]:
function get_target(df)
    return df[:, target]
end

get_target (generic function with 1 method)

In [15]:
function compute_results(pred, test)
    # compute the mean absolute error
    mae = Metrics.mae(pred, test)
    # compute the mean squared error
    mse = Metrics.mse(pred, test)
    # compute the root mean squared error
    rmse = sqrt.(mse)
    # compute the r2
    osr2 = Metrics.r2_score(pred, test)

    # turn into dataframe
    results = DataFrame(mae = mae, mse = mse, rmse = rmse, osr2 = osr2)

    # print the results
    display(results)
end

compute_results (generic function with 1 method)

In [16]:
function check_prediction_value(pred, test)

    # set plot dimensions
    plot_size = (800, 400)
    
    histogram(pred, label = "pred", color = "red", seriesalpha = 0.5)
    display(histogram!(test, label = "test", color = "blue", seriesalpha = 0.5))

end

check_prediction_value (generic function with 1 method)

# Approach 0: What truly happened in the last transfer window

In [97]:
# useful to compare our model with what truly happens
transfers_club = transfers[transfers[:, "club_name"] .== club_choosen_transferName, :]
transfers_club = transfers_club[completecases(transfers_club), :];

In [98]:
transfers_in = transfers_club[transfers_club[:, "transfer_movement"] .== "in", :]

Row,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
,String,String,Float64?,String31,String31,String31?,String3,String7,Float64?,String31,Int64,String15
1,FC Barcelona,Raphinha,25.0,Right Winger,Leeds,€58.00m,in,summer,58.0,Primera Division,2022,2022/2023
2,FC Barcelona,Jules Koundé,23.0,Centre-Back,Sevilla FC,€50.00m,in,summer,50.0,Primera Division,2022,2022/2023
3,FC Barcelona,Robert Lewandowski,33.0,Centre-Forward,Bayern Munich,€45.00m,in,summer,45.0,Primera Division,2022,2022/2023
4,FC Barcelona,Marcos Alonso,31.0,Left-Back,Chelsea,free transfer,in,summer,0.0,Primera Division,2022,2022/2023
5,FC Barcelona,Héctor Bellerín,27.0,Right-Back,Arsenal,free transfer,in,summer,0.0,Primera Division,2022,2022/2023
6,FC Barcelona,Andreas Christensen,26.0,Centre-Back,Chelsea,free transfer,in,summer,0.0,Primera Division,2022,2022/2023
7,FC Barcelona,Franck Kessié,25.0,Central Midfield,AC Milan,free transfer,in,summer,0.0,Primera Division,2022,2022/2023


In [99]:
transfers_out = transfers_club[transfers_club[:, "transfer_movement"] .== "out", :]

Row,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
,String,String,Float64?,String31,String31,String31?,String3,String7,Float64?,String31,Int64,String15
1,FC Barcelona,Philippe Coutinho,30.0,Left Winger,Aston Villa,€20.00m,out,summer,20.0,Primera Division,2022,2022/2023
2,FC Barcelona,Pierre-Emerick Aubameyang,33.0,Centre-Forward,Chelsea,€12.00m,out,summer,12.0,Primera Division,2022,2022/2023
3,FC Barcelona,Francisco Trincão,22.0,Right Winger,Sporting CP,Loan fee:€3.00m,out,summer,3.0,Primera Division,2022,2022/2023
4,FC Barcelona,Dani Alves,39.0,Right-Back,UNAM Pumas,free transfer,out,summer,0.0,Primera Division,2022,2022/2023
5,FC Barcelona,Miralem Pjanic,32.0,Central Midfield,Sharjah FC,free transfer,out,summer,0.0,Primera Division,2022,2022/2023
6,FC Barcelona,Martin Braithwaite,31.0,Centre-Forward,Espanyol,free transfer,out,summer,0.0,Primera Division,2022,2022/2023
7,FC Barcelona,Neto,33.0,Goalkeeper,Bournemouth,free transfer,out,summer,0.0,Primera Division,2022,2022/2023
8,FC Barcelona,Riqui Puig,22.0,Central Midfield,Los Angeles,free transfer,out,summer,0.0,Primera Division,2022,2022/2023
9,FC Barcelona,Óscar Mingueza,23.0,Centre-Back,Celta de Vigo,free transfer,out,summer,0.0,Primera Division,2022,2022/2023


In [104]:
revenue =  sum(transfers_out[:, "fee_cleaned"]) - sum(transfers_in[:, "fee_cleaned"])

-118.0

# Approach 1: Regress and compare

In [53]:
function regress_and_compare(data_club, data_rest, transfers_club)

    # get parameters
    n, N_c, N_min, N_max, B_max, S_max, p_o, r_o, v_o, m_o, a_o, p_c, r_c, v_c, m_c, a_c, 𝛅, 𝛾 = get_parameters(data_club, data_rest, transfers_club)

    # define model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(model, "OutputFlag" => 0, "TimeLimit" => 60)

    # define variables
    @variable(model, b[i = 1:n], Bin) # 1 if we buy player i, 0 otherwise
    @variable(model, s[j = 1:k], Bin) # 1 if we sell player j, 0 otherwise

    # define objective
    @objective(model, Max, sum(s[j] * v_c[j] for j = 1:k) - sum(b[i] * v_o[i] for i = 1:n))

    # define constraints
    @constraint(model, sum((1 - s[j]) * m_c[j] for j = 1:k) + sum(b[i] * m_o[i] for i = 1:n) >= (sum(m_c[j] for j = 1:k)/k + 𝛅) * (k + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:k)))
    @constraint(model, sum(b[i] for i = 1:n) <= B_max) # maximum number of players that can be bought
    @constraint(model, sum(s[j] for j = 1:k) <= S_max) # maximum number of players that can be sold
    @constraint(model, N_min <= N_c + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:k) <= N_max) # number of players in the club after the transfer window
    @constraint(model, sum((1 - s[j]) * a_c[j] for j = 1:k) + sum(b[i] * a_o[i] for i = 1:n) <= (sum(a_c[j] for j = 1:k)/k + 𝛾) * (k + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:k))) # average age of the team

    # solve model
    optimize!(model)

    return objective_value(model), value.(b), value.(s)
end

regress_and_compare (generic function with 2 methods)

In [54]:
obj, b, s = regress_and_compare(data_club, data_rest, transfers_club);

In [56]:
# get the players that we buy and sell
buy = data_rest[b .== 1, :]

Row,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_D,position_G,position_M,attack_work_rate,defense_work_rate
,String31,Int64,Int64,Float64,Float64,Int64,Int64,Int64,String,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,Sergio Ramos,88,88,2.4e7,115000.0,35,184,82,Paris Saint-Germain,1.0,2021,2023.0,1,3,3,4,4.44e7,70.0,70.0,76.0,74.0,88.0,84.0,66,65,92,82,69,65,74,76,83,83,71,69,77,92,66,79,93,78,84,64,90,87,73,71,92,88,84,89,91,11,8,9,7,11,0,1,0,0,3,2
2,Rosberto Dourado,81,81,2.7e7,26000.0,21,175,70,Sport Club Corinthians Paulista,1.0,2021,2024.0,1,4,2,1,5.13e7,77.0,57.0,74.0,77.0,82.0,68.0,61,53,70,82,57,77,54,57,78,76,75,78,72,80,77,60,40,91,56,59,75,82,58,77,77,81,83,86,80,15,14,14,10,9,0,0,0,1,3,2
3,Welington Dano,81,81,2.75e7,31000.0,21,178,69,Clube Atlético Mineiro,1.0,2021,2024.0,0,4,4,1,4.95e7,79.0,54.0,76.0,79.0,75.0,74.0,82,51,69,78,47,77,83,52,71,78,78,80,83,80,81,55,71,89,68,53,69,77,72,75,61,82,70,77,81,16,15,15,7,7,0,1,0,0,2,2
4,B. Kamara,80,86,3.1e7,30000.0,21,184,68,Olympique de Marseille,1.0,2021,2022.0,1,3,3,2,6.51e7,74.0,50.0,72.0,73.0,79.0,78.0,64,39,73,79,28,70,59,47,78,78,74,74,66,73,67,66,74,78,76,63,83,83,43,74,63,81,76,81,80,8,12,8,6,11,0,0,0,1,2,3


In [57]:
sell = data_club[s .== 1, :]

Row,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_D,position_G,position_M,attack_work_rate,defense_work_rate
,String31,Int64,Int64,Float64,Float64,Int64,Int64,Int64,String,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,F. de Jong,87,92,1.195e8,210000.0,24,180,74,FC Barcelona,1.0,2021,2026.0,1,3,4,3,2.539e8,81.0,69.0,85.0,88.0,77.0,78.0,78,72,72,91,70,88,84,64,87,89,80,82,88,88,79,68,76,90,74,66,75,82,77,86,45,90,76,76,77,7,13,10,10,9,0,0,0,1,3,2
2,Matheus Pereira,68,76,2.7e6,35000.0,23,181,68,FC Barcelona,1.0,2021,2025.0,0,3,4,1,6.1e6,69.0,60.0,65.0,74.0,35.0,58.0,60,62,52,67,66,75,73,64,63,74,68,69,78,70,69,58,59,69,55,57,50,33,62,67,62,73,25,40,38,9,13,7,9,10,0,0,0,1,2,2
3,Álex Collado,67,79,2.5e6,33000.0,22,177,66,FC Barcelona,1.0,2021,2023.0,0,3,3,1,5.6e6,63.0,65.0,70.0,69.0,56.0,39.0,70,64,51,68,53,71,73,75,64,68,64,63,69,66,68,68,43,34,38,70,48,53,62,74,63,67,59,56,57,10,9,12,12,11,1,0,0,0,2,2


# Useless because objective function is linear

## Approach 2: KNN

In [35]:
num_neighbors = 6
;

In [84]:
v = new_data[: , "value_eur"]; # find value for all players

In [85]:
# knn is not used for predicting the transfer fee, but for finding the nearest players to the one being considered
function find_nearest_neighbors(new_data)

    new_data = new_data[:, names(new_data) .!= "short_name"]
    new_data = new_data[: , names(new_data) .!= "club_name"]
    
    # train k-Nearest Neighbors Model
    df = vcat(train_valid, test)

    xDataTraining = Matrix(get_features(df))
    yDataTraining = get_target(df)

    xDataNew = Matrix(new_data)

    kdtree = KDTree(xDataTraining'; leafsize = 10)

    # indices returns for each player the indices of the k nearest neighbors
    indices, distance = knn(kdtree, xDataNew', num_neighbors)

    return indices
end

find_nearest_neighbors (generic function with 1 method)

In [83]:
function k_near_neigh(data_club, data_rest, transfers_club, num_neighbors)

    # get parameters
    n, N_c, N_min, N_max, B_max, S_max, p_o, r_o, v_o, m_o, a_o, p_c, r_c, v_c, m_c, a_c, 𝛅, 𝛾 = get_parameters(data_club, data_rest, transfers_club)

    # find nearest neighbors
    indices_club = find_nearest_neighbors(data_club)
    indices_rest = find_nearest_neighbors(data_rest)

    # define model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(model, "OutputFlag" => 0, "TimeLimit" => 60)

    # define variables
    @variable(model, b[i = 1:n], Bin) # 1 if we buy player i, 0 otherwise
    @variable(model, s[j = 1:N_c], Bin) # 1 if we sell player j, 0 otherwise

    # define objective
    @objective(model, Max, sum(sum(s[j] * v[l] for l in indices_club[j]) / num_neighbors for j = 1:N_c) - 
            sum(sum(b[i] * v[l] for l in indices_rest[i]) / num_neighbors for i = 1:n))

    # define constraints
    @constraint(model, sum((1 - s[j]) * m_c[j] for j = 1:N_c) + sum(b[i] * m_o[i] for i = 1:n) >= (sum(m_c[j] for j = 1:N_c)/N_c + 𝛅) * (N_c + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:N_c)))
    @constraint(model, sum(b[i] for i = 1:n) <= B_max) # maximum number of players that can be bought
    @constraint(model, sum(s[j] for j = 1:N_c) <= S_max) # maximum number of players that can be sold
    @constraint(model, N_min <= N_c + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:N_c) <= N_max) # number of players in the club after the transfer window
    @constraint(model, sum((1 - s[j]) * a_c[j] for j = 1:N_c) + sum(b[i] * a_o[i] for i = 1:n) <= (sum(a_c[j] for j = 1:N_c)/N_c + 𝛾) * (N_c + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:N_c))) # average age of the team

    # solve model
    optimize!(model)

    return objective_value(model), value.(b), value.(s)
end

k_near_neigh (generic function with 2 methods)

In [80]:
obj_knn, b_knn, s_knn = k_near_neigh(data_club, data_rest, transfers_club, num_neighbors);

1234567

In [81]:
# get the players that we buy and sell
buy_knn = data_rest[b_knn .== 1, :]

Row,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_D,position_G,position_M,attack_work_rate,defense_work_rate
,String31,Int64,Int64,Float64,Float64,Int64,Int64,Int64,String,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,K. Mbappé,91,95,1.94e8,230000.0,22,182,73,Paris Saint-Germain,1.0,2021,2022.0,1,4,5,4,3.735e8,97.0,88.0,80.0,92.0,36.0,77.0,78,93,72,85,83,93,80,69,71,91,97,97,92,93,83,86,78,88,77,82,62,38,92,82,79,88,26,34,32,13,5,7,11,6,1,0,0,0,3,1


In [82]:
sell_knn = data_club[s_knn .== 1, :]

Row,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_D,position_G,position_M,attack_work_rate,defense_work_rate
,String31,Int64,Int64,Float64,Float64,Int64,Int64,Int64,String,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,S. Umtiti,80,80,1.95e7,125000.0,27,182,75,FC Barcelona,1.0,2021,2023.0,0,3,2,3,4.0e7,65.0,62.0,68.0,67.0,80.0,78.0,61,51,78,76,65,64,72,62,73,73,64,65,58,78,64,83,81,64,84,70,80,81,51,58,59,74,80,80,79,15,10,14,12,15,0,1,0,0,1,2
2,Álex Collado,67,79,2.5e6,33000.0,22,177,66,FC Barcelona,1.0,2021,2023.0,0,3,3,1,5.6e6,63.0,65.0,70.0,69.0,56.0,39.0,70,64,51,68,53,71,73,75,64,68,64,63,69,66,68,68,43,34,38,70,48,53,62,74,63,67,59,56,57,10,9,12,12,11,1,0,0,0,2,2
3,Gavi,66,85,2.1e6,4000.0,16,173,68,FC Barcelona,1.0,2021,2023.0,1,3,2,1,5.6e6,66.0,53.0,67.0,69.0,50.0,48.0,59,55,38,73,46,67,65,47,69,70,68,64,77,64,78,49,48,52,44,53,54,58,59,70,51,68,49,53,44,8,10,11,7,13,0,0,0,1,2,2


## Approach 3: CART and OCT